In [27]:
from datetime import date
import numpy as np
import open3d as o3d

today = date.today()
#Normals Vis
import open3d as o3d
from pyntcloud import PyntCloud

pcd = o3d.io.read_point_cloud("/home/sashawald/Documents/shaving_code-main/clouds/2022-06-03.ply")

#Visualize Original
#print(pcd)
#o3d.visualization.draw_geometries([pcd])
R = np.identity(3)  
extent = np.ones(3)/.32 # trying to create a bounding box below 1 unit
center = np.zeros(3) 
obb = o3d.geometry.OrientedBoundingBox(center,R,extent)
pcd = pcd.crop(obb)
dpcd = pcd.voxel_down_sample(voxel_size=0.035)
#use draw_geometries if you don't need to edit the result
o3d.visualization.draw_geometries_with_editing([dpcd])
#cloud = PyntCloud.from_instance("open3d", pcd)
#scene = cloud.plot(return_scene=True, position=(1,1,1))


In [28]:
#Code  for picking specific points: shift+left click to select, shift+right click to deselect
#close visualization window to return selected points
import numpy as np
from LineMesh import *

vis = o3d.visualization.VisualizerWithEditing()
vis.create_window()
vis.add_geometry(dpcd)
vis.run()  
vis.destroy_window()
pdata = vis.get_picked_points()


[Open3D INFO] Picked point #780 (-0.16, -0.1, -1.5) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #378 (0.3, -0.092, -1.4) to add in queue.


In [29]:
import networkx as nx
G = nx.Graph()

pcd_tree = o3d.geometry.KDTreeFlann(dpcd)
count=0
for vertex in dpcd.points:
    [k, idx, _] = pcd_tree.search_knn_vector_3d(vertex, 5)
    for v_curr in idx:
        current_vertex=dpcd.points[v_curr]
        if not (count==v_curr):
            dist = np.sqrt(np.sum((vertex-current_vertex)**2, axis=0))
            G.add_edge(count,v_curr,weight=dist)
    count=count+1

path=(nx.dijkstra_path(G,pdata[0],pdata[1]))
foundpcd = dpcd.select_by_index(path)
foundpcd.paint_uniform_color([1, 0, 0])
dpcd.paint_uniform_color([0,1,0])
o3d.visualization.draw_geometries([dpcd,foundpcd])